**This notebook is of my submition to the Assigment project for freecodecamp**

https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/linear-regression-health-costs-calculator

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

### **Loading the Dataset**

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')

### **Data Preprocessing**

In [ ]:
dataset.head()

In [ ]:
dataset.isna().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le_sex = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()

In [ ]:
dataset['sex'] = le_sex.fit_transform(dataset['sex'])
dataset['smoker'] = le_smoker.fit_transform(dataset['smoker'])
dataset['region'] = le_region.fit_transform(dataset['region'])

In [ ]:
dataset.head()

### **Splitting the data**

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset.drop(['expenses'], axis=1)
y = dataset['expenses']

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
scaler = StandardScaler()
numerical_features = ['age', 'bmi', 'children']
train_dataset[numerical_features] = scaler.fit_transform(train_dataset[numerical_features])
test_dataset[numerical_features] = scaler.transform(test_dataset[numerical_features])

### **Creating the Model**

In [ ]:
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(train_dataset.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

In [ ]:
history = model.fit(train_dataset, train_labels, epochs=200, validation_split=0.1)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print(f"MAE: {mae:.2f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Training and Validation Loss (MSE)
ax1.plot(history.history['loss'], label='Training Loss (MSE)')
ax1.plot(history.history['val_loss'], label='Validation Loss (MSE)')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (MSE)')
ax1.legend()
ax1.grid(True)

# Plot 2: Training and Validation MAE
ax2.plot(history.history['mae'], label='Training MAE')
ax2.plot(history.history['val_mae'], label='Validation MAE')
ax2.set_title('Training and Validation MAE')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MAE')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
predictions = model.predict(test_dataset).flatten()
plt.scatter(test_labels, predictions, label='Predicted vs Actual')
plt.plot([test_labels.min(), test_labels.max()], [test_labels.min(), test_labels.max()], 'r--')
plt.xlabel('Actual Expenses')
plt.ylabel('Predicted Expenses')
plt.legend()
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
